In [10]:
import pandas as pd
import numpy as np
from sklearn import model_selection
from sklearn import metrics
from sklearn import preprocessing
from sklearn.cluster import AffinityPropagation, KMeans, MeanShift, SpectralClustering, AgglomerativeClustering, DBSCAN, OPTICS
import matplotlib.pyplot as plt
from sksurv.metrics import brier_score, concordance_index_censored, cumulative_dynamic_auc
import matplotlib.pyplot as plt
from sksurv.util import Surv
from lifelines.fitters.kaplan_meier_fitter import KaplanMeierFitter
from sksurv.nonparametric import kaplan_meier_estimator

In [11]:
DAT = []
for i in range(20):
    dat = pd.read_csv(f'C:/Users/ndong/OneDrive - Texas Tech University/Documents/Scalable Survival-DP Package/Simulation/Original Copy of Simulation/Simulation 3 - small std/simulated data 3_{i+1}.csv', index_col=0)
    DAT.append(dat)

In [12]:
ind_train = [0,1,2,3,4,5,6,8,9,10,11,12,13,14,15,16]
ind_test = [7,17,18,19]
Train = pd.concat([DAT[i] for i in ind_train], ignore_index=True)
Test = pd.concat([DAT[i] for i in ind_test], ignore_index=True)

In [13]:
Train['t'] = np.exp(Train['Y'])
Test['t'] = np.exp(Test['Y'])
Training = Train.drop(columns=['z_x', 'z_y', 'y_mu', 'y_sigma', 'Y'])
Testing = Test.drop(columns=['z_x', 'z_y', 'y_mu', 'y_sigma', 'Y'])
Training.columns

Index(['x_1', 'x_2', 'x_3', 'x_4', 'x_5', 'x_6', 'x_7', 'x_8', 'x_9', 'x_10',
       'x_11', 'x_12', 'x_13', 'x_14', 'x_15', 'x_16', 'x_17', 'x_18', 'x_19',
       'x_20', 'x_21', 'x_22', 'x_23', 'x_24', 'x_25', 'x_26', 'x_27', 'x_28',
       'x_29', 'x_30', 'x_31', 'x_32', 'x_33', 'x_34', 'x_35', 'x_36', 'x_37',
       'x_38', 'x_39', 'x_40', 'x_41', 'x_42', 'x_43', 'x_44', 'x_45', 'x_46',
       'x_47', 'x_48', 'x_49', 'x_50', 'c_y', 't'],
      dtype='object')

In [14]:
True_train = Train['z_y']
True_test = Test['z_y']

Kmeans

In [23]:
cluster_range = [2,3,4,5,6]
train_RI = []
test_RI = []
train_MI = []
test_MI = []
train_ARI = []
test_ARI = []
train_AMI = []
test_AMI = []
train_HOMO = []
test_HOMO = []
train_COM = []
test_COM = []
train_VM = []
test_VM = []
test_FM = []
train_FM = []

for i in cluster_range:
    km = KMeans(n_clusters=i, random_state=555)
    km.fit(Training)
    pred_train = km.labels_
    pred_test = km.predict(Testing)
    train_ari = metrics.adjusted_rand_score(True_train, pred_train)
    test_ari = metrics.adjusted_rand_score(True_test, pred_test)
    train_ami = metrics.adjusted_mutual_info_score(True_train, pred_train)
    test_ami = metrics.adjusted_mutual_info_score(True_test, pred_test)
    train_ri = metrics.rand_score(True_train, pred_train)
    test_ri = metrics.rand_score(True_test, pred_test)
    train_mi = metrics.mutual_info_score(True_train, pred_train)
    test_mi = metrics.mutual_info_score(True_test, pred_test)
    train_hcv = metrics.homogeneity_completeness_v_measure(True_train, pred_train)
    test_hcv = metrics.homogeneity_completeness_v_measure(True_test, pred_test)
    train_ARI.append(train_ari)
    test_ARI.append(test_ari)
    train_AMI.append(train_ami)
    test_AMI.append(test_ami)
    train_HOMO.append(train_hcv[0])
    test_HOMO.append(test_hcv[0])
    train_COM.append(train_hcv[1])
    test_COM.append(test_hcv[1])
    train_VM.append(train_hcv[2])
    test_VM.append(test_hcv[2])
    train_RI.append(train_ri)
    test_RI.append(test_ri)
    train_MI.append(train_mi)
    test_MI.append(test_mi)
    train_fm = metrics.fowlkes_mallows_score(True_train, pred_train)
    test_fm = metrics.fowlkes_mallows_score(True_test, pred_test)
    test_FM.append(test_fm)
    train_FM.append(train_fm)

KM_metrics = pd.DataFrame({'train RI': train_RI, 'test RI': test_RI, 'train MI': train_MI, 'test MI': test_MI, 'train ARI': train_ARI, 'test ARI': test_ARI, 'train AMI': train_AMI, 'test AMI': test_AMI, 'train Homogeneity': train_HOMO, 'test Homogeneity': test_HOMO, 'train Completeness': train_COM, 'test Completeness': test_COM, 'train V-measure': train_VM, 'test V-measure': test_VM, 'train FM': train_FM, 'test FM': test_FM}, index=cluster_range)
KM_metrics

    

,train RI,test RI,train MI,test MI,train ARI,test ARI,train AMI,test AMI,train Homogeneity,test Homogeneity,train Completeness,test Completeness,train V-measure,test V-measure,train FM,test FM
2,0.754971,0.755056,0.539522,0.536133,0.521439,0.521891,0.614714,0.611055,0.491106,0.488023,0.821670,0.817826,0.614769,0.611278,0.738497,0.739004
3,0.781199,0.782689,0.639488,0.644481,0.548022,0.550456,0.642052,0.645369,0.582101,0.586649,0.715992,0.718036,0.642142,0.645727,0.729907,0.730817
4,0.799396,0.797496,0.673458,0.675254,0.565154,0.560821,0.618745,0.617599,0.613022,0.614660,0.624844,0.621637,0.618876,0.618129,0.722135,0.719169
5,0.786010,0.785477,0.676993,0.680061,0.503211,0.499984,0.558228,0.557662,0.616240,0.619036,0.510504,0.508576,0.558411,0.558396,0.659351,0.656051
6,0.786634,0.784613,0.690677,0.686214,0.495387,0.490493,0.549025,0.543528,0.628696,0.624637,0.487630,0.482512,0.549250,0.544452,0.649245,0.645721


Affinity propagation

In [31]:
train_ARI = []
test_ARI = []
train_AMI = []
test_AMI = []
train_HOMO = []
test_HOMO = []
train_COM = []
test_COM = []
train_VM = []
test_VM = []

AP = AffinityPropagation(max_iter=2000, random_state=888)
AP.fit(Training)
pred_train = AP.labels_
pred_test = AP.predict(Testing)
train_ari = metrics.adjusted_rand_score(True_train, pred_train)
test_ari = metrics.adjusted_rand_score(True_test, pred_test)
train_ami = metrics.adjusted_mutual_info_score(True_train, pred_train)
test_ami = metrics.adjusted_mutual_info_score(True_test, pred_test)
train_hcv = metrics.homogeneity_completeness_v_measure(True_train, pred_train)
test_hcv = metrics.homogeneity_completeness_v_measure(True_test, pred_test)
train_ARI.append(train_ari)
test_ARI.append(test_ari)
train_AMI.append(train_ami)
test_AMI.append(test_ami)
train_HOMO.append(train_hcv[0])
test_HOMO.append(test_hcv[0])
train_COM.append(train_hcv[1])
test_COM.append(test_hcv[1])
train_VM.append(train_hcv[2])
test_VM.append(test_hcv[2])

AP_metrics = pd.DataFrame({'train ARI': train_ARI, 'test ARI': test_ARI, 'train AMI': train_AMI, 'test AMI': test_AMI, 'train Homogeneity score': train_HOMO, 'test Homogeneity score': test_HOMO, 'train Completeness score': train_COM, 'test Completeness score': test_COM, 'train V-measure': train_VM, 'test V-measure score': test_VM})
AP_metrics

c:\Users\ndong\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\cluster\_affinity_propagation.py:142: ConvergenceWarning: Affinity propagation did not converge, this model may return degenerate cluster centers and labels.
  warnings.warn(


,train ARI,test ARI,train AMI,test AMI,train Homogeneity score,test Homogeneity score,train Completeness score,test Completeness score,train V-measure,test V-measure score
0,0.430174,0.428875,0.277112,0.327432,0.902581,0.897716,0.230378,0.261957,0.367065,0.405568


In [ ]:
train_ARI = []
test_ARI = []
train_AMI = []
test_AMI = []
train_HOMO = []
test_HOMO = []
train_COM = []
test_COM = []
train_VM = []
test_VM = []

AP = AffinityPropagation(random_state=555)
AP.fit(Training)
pred_train = AP.labels_
pred_test = AP.predict(Testing)
train_ari = metrics.rand_score(True_train, pred_train)
test_ari = metrics.rand_score(True_test, pred_test)
train_ami = metrics.mutual_info_score(True_train, pred_train)
test_ami = metrics.mutual_info_score(True_test, pred_test)
train_hcv = metrics.homogeneity_completeness_v_measure(True_train, pred_train)
test_hcv = metrics.homogeneity_completeness_v_measure(True_test, pred_test)
train_ARI.append(train_ari)
test_ARI.append(test_ari)
train_AMI.append(train_ami)
test_AMI.append(test_ami)
train_HOMO.append(train_hcv[0])
test_HOMO.append(test_hcv[0])
train_COM.append(train_hcv[1])
test_COM.append(test_hcv[1])
train_VM.append(train_hcv[2])
test_VM.append(test_hcv[2])

AP_metrics = pd.DataFrame({'train RI': train_ARI, 'test RI': test_ARI, 'train MI': train_AMI, 'test MI': test_AMI, 'train Homogeneity score': train_HOMO, 'test Homogeneity score': test_HOMO, 'train Completeness score': train_COM, 'test Completeness score': test_COM, 'train V-measure': train_VM, 'test V-measure score': test_VM})
AP_metrics

Mean shift

In [18]:
cluster_range = [2,3,4,5,6]
train_ARI = []
test_ARI = []
train_AMI = []
test_AMI = []
train_HOMO = []
test_HOMO = []
train_COM = []
test_COM = []
train_VM = []
test_VM = []
train_RI = []
test_RI = []
train_MI = []
test_MI = []

for i in cluster_range:
    MS = MeanShift(bandwidth=i)
    MS.fit(Training)
    pred_train = MS.labels_
    pred_test = MS.predict(Testing)
    train_ari = metrics.adjusted_rand_score(True_train, pred_train)
    test_ari = metrics.adjusted_rand_score(True_test, pred_test)
    train_ami = metrics.adjusted_mutual_info_score(True_train, pred_train)
    test_ami = metrics.adjusted_mutual_info_score(True_test, pred_test)
    train_ri = metrics.rand_score(True_train, pred_train)
    test_ri = metrics.rand_score(True_test, pred_test)
    train_mi = metrics.mutual_info_score(True_train, pred_train)
    test_mi = metrics.mutual_info_score(True_test, pred_test)
    train_hcv = metrics.homogeneity_completeness_v_measure(True_train, pred_train)
    test_hcv = metrics.homogeneity_completeness_v_measure(True_test, pred_test)
    train_ARI.append(train_ari)
    test_ARI.append(test_ari)
    train_AMI.append(train_ami)
    test_AMI.append(test_ami)
    train_HOMO.append(train_hcv[0])
    test_HOMO.append(test_hcv[0])
    train_COM.append(train_hcv[1])
    test_COM.append(test_hcv[1])
    train_VM.append(train_hcv[2])
    test_VM.append(test_hcv[2])
    train_RI.append(train_ri)
    test_RI.append(test_ri)
    train_MI.append(train_mi)
    test_MI.append(test_mi)

MS_metrics = pd.DataFrame({'train ARI': train_ARI, 'test ARI': test_ARI, 'train AMI': train_AMI, 'test AMI': test_AMI, 'train RI': train_RI, 'test RI': test_RI, 'train MI': train_MI, 'test MI': test_MI, 'train Homogeneity score': train_HOMO, 'test Homogeneity score': test_HOMO, 'train Completeness score': train_COM, 'test Completeness score': test_COM, 'train V-measure': train_VM, 'test V-measure score': test_VM})
MS_metrics

,train ARI,test ARI,train AMI,test AMI,train RI,test RI,train MI,test MI,train Homogeneity score,test Homogeneity score,train Completeness score,test Completeness score,train V-measure,test V-measure score
0,0.042270,0.106770,0.042007,0.122722,0.677410,0.694367,1.098586,1.098580,1.0,1.000000,0.134214,0.174546,0.236664,0.297215
1,0.311661,0.309541,0.207732,0.218163,0.751209,0.750747,1.098586,1.097325,1.0,0.998857,0.184674,0.208563,0.311772,0.345074
2,0.339700,0.337382,0.226128,0.240431,0.759514,0.758976,1.098586,1.097224,1.0,0.998765,0.191271,0.215165,0.321121,0.354056
3,0.361678,0.360872,0.356756,0.331095,0.766114,0.766018,1.098586,1.097325,1.0,0.998857,0.235121,0.241894,0.380725,0.389470
4,0.382925,0.380028,0.389161,0.365190,0.772570,0.771828,1.098586,1.097325,1.0,0.998857,0.252139,0.256546,0.402734,0.408240


Hierarchical Clustering

In [25]:
cluster_range = [2,3,4,5,6]
train_ARI = []
test_ARI = []
train_AMI = []
test_AMI = []
train_HOMO = []
test_HOMO = []
train_COM = []
test_COM = []
train_VM = []
test_VM = []
train_RI = []
test_RI = []
train_MI = []
test_MI = []
true=np.concatenate([True_train, True_test])
train_FM = []

for i in cluster_range:
    HC = AgglomerativeClustering(n_clusters=i)
    HC.fit(pd.concat([Training, Testing]))
    pred = HC.labels_
    ari = metrics.adjusted_rand_score(true, pred)
    ri = metrics.rand_score(true,pred)
    ami = metrics.adjusted_mutual_info_score(true, pred)
    mi = metrics.mutual_info_score(true,pred)
    hcv = metrics.homogeneity_completeness_v_measure(true, pred)
    fm = metrics.fowlkes_mallows_score(true, pred)
    train_ARI.append(ari)
    train_AMI.append(ami)
    train_RI.append(ri)
    train_MI.append(mi)
    train_HOMO.append(hcv[0])
    train_COM.append(hcv[1])
    train_VM.append(hcv[2])
    train_FM.append(fm)

HC_metrics = pd.DataFrame({'RI': train_RI, 'MI': train_MI,'ARI': train_ARI, 'AMI': train_AMI, 'Homogeneity score': train_HOMO, 'Completeness score': train_COM, 'V-measure': train_VM, 'FM': train_FM}, index=cluster_range)
HC_metrics

,RI,MI,ARI,AMI,Homogeneity score,Completeness score,V-measure,FM
2,0.773004,0.624422,0.563037,0.720987,0.568387,0.985719,0.721019,0.770403
3,0.808999,0.704081,0.588448,0.668941,0.640898,0.699687,0.669003,0.739225
4,0.803981,0.706653,0.575656,0.650751,0.643239,0.658637,0.650847,0.729368
5,0.798390,0.712457,0.518999,0.564767,0.648523,0.500387,0.564905,0.664235
6,0.793064,0.717768,0.503241,0.551143,0.653357,0.476842,0.551316,0.651813


DBSCAN

In [26]:
cluster_range = [2,3,4,5,6]
train_ARI = []
test_ARI = []
train_AMI = []
test_AMI = []
train_RI = []
train_MI = []
train_HOMO = []
test_HOMO = []
train_COM = []
test_COM = []
train_VM = []
test_VM = []
train_FM = []
true=np.concatenate([True_train, True_test])

OP = OPTICS()
OP.fit(pd.concat([Training, Testing]))
pred = OP.labels_
ari = metrics.adjusted_rand_score(true, pred)
ami = metrics.adjusted_mutual_info_score(true, pred)
ri = metrics.rand_score(true, pred)
mi = metrics.mutual_info_score(true, pred)
hcv = metrics.homogeneity_completeness_v_measure(true, pred)
fm = metrics.fowlkes_mallows_score(true, pred)
train_ARI.append(ari)
train_AMI.append(ami)
train_RI.append(ri)
train_MI.append(mi)
train_HOMO.append(hcv[0])
train_COM.append(hcv[1])
train_VM.append(hcv[2])
train_FM.append(fm)

OP_metrics = pd.DataFrame({'RI': train_RI, 'MI': train_MI,'ARI': train_ARI, 'AMI': train_AMI, 'Homogeneity score': train_HOMO, 'Completeness score': train_COM, 'V-measure': train_VM, 'FM': train_FM})
OP_metrics

,RI,MI,ARI,AMI,Homogeneity score,Completeness score,V-measure,FM
0,0.792914,0.778813,0.533456,0.483398,0.708924,0.375537,0.490985,0.688572
